### Extended GCD (Bezout Coefficient)

In [2]:
def extended_gcd(a, b):
    if b == 0:
        return a, 1, 0  # gcd(a, 0) = a, with coefficients (1, 0)

    gcd, x1, y1 = extended_gcd(b, a % b)  # Recursive call

    # Update coefficients
    x = y1
    y = x1 - (a // b) * y1

    return gcd, x, y


# -----------
# Example usage
a, b = 11, 19
gcd, x, y = extended_gcd(a, b)
print(f"GCD: {gcd}, x: {x}, y: {y}")


GCD: 1, x: 7, y: -4


### Message Encode and Decode

In [3]:
def convert_string_to_numbers(s, q=101 ):
    lst = [*s] # unpack string into a list
    n = len(lst)
    r = 0 if n%5 == 0 else 5 - (n%5)
    # chr(31) is a null character that we use to pad the message to make its size a multiple of the block length (5)
    lst += [chr(31)]*r # pad it with a special character ascii 31
    n = len(lst)
    
    msg = []
    # run through the characters 5 at a time
    for i in range(0, n, 5):
        block = lst[i:i+5]
        c = [ord(k)-31 for k in block] # subtract 31 from ascii values
        m = 0
        for i in range(4, -1, -1): # convert from base-q to decimal
            m = m * q + c[i]
        msg.append(m) # append number to message
    return msg


msg = convert_string_to_numbers('Hello, this is a message that needs to be encoded!!')
for j in msg:
    print(j)


def convert_numbers_to_strings(msg, q=101 ):
    n = len(msg)
    assert n > 0, 'Nonempty list of numbers required'
    blk_len = 5 # hard coded block length
    codes = []
    for k in msg: # each number is a decimal representation of a 5 digit base q number
        for i in range(5): # convert it back into base q
            r = k%q
            if r >= 1:
                codes.append(chr(r + 31)) # convert from ascii back into character but add 31
            k = k//q
    # convert from sequence of chars back into a string
    return ''.join(codes)

convert_numbers_to_strings(msg)

8404957845
7776548846
191360744
8813990599
176922693
192316710
8812885672
6973901834
7158215288
279932690
2


'Hello, this is a message that needs to be encoded!!'

### Encrypt and Decrypt

In [4]:
def encrypt_naive(M, e, n):
    p = 1
    for i in range(e):
        p = (p * M) % n
    return p

encrypt_naive(9, 13, 77) # encrypt 9 with the public key

58

In [5]:
encrypt_naive(58, 37, 77)

9

Generating the public and private key is the trick here. 

In [6]:
def generate_rsa_keys(p, q):
    assert p < q
    n = p * q
    phi = (p-1)*(q-1)
    e = None
    d = None
    for e in range(p//3, p):
        (i, s, t) = extended_gcd(phi, e)
        if i == 1:
            if t < 0:
                d = phi + t
            else:
                d = t
            break
    return (n, e, d)

generate_rsa_keys(7, 13)

(91, 5, 29)

In [7]:
(n, e, d) = generate_rsa_keys(735193, 875491)
print(n)
print(e)
print(d)

643654854763
245071
478235009071


In [8]:
def rsa_encode(msg_list, e, n):
    rsa_lst = [encrypt_naive(M, e, n) for M in msg_list]
    return rsa_lst


orig_msg = convert_string_to_numbers('Hello, I have to say something secret!!')
print('Original message:')
print(orig_msg)
print(f'Public Key: {e, n}')
msg_rsa = rsa_encode(orig_msg, e, n)
print('RSA encoded')
print(msg_rsa)

Original message:
[8404957845, 7597868130, 8846887309, 9434293021, 7365416113, 7574504983, 8707796306, 2089659]
Public Key: (245071, 643654854763)
RSA encoded
[46264071735, 544973177667, 185197985834, 152593409466, 416635991589, 482835244299, 65259378513, 220105851236]


In [9]:
# If we tried to read the encrypted message, we get junk back.
print('RSA encoded string is')
convert_numbers_to_strings(msg_rsa)

RSA encoded string is


"2cLZG>>['u)FAg]=`aFS]ann_98,\x7f}T(*,4r_X0~"

In [10]:
msg_decode = rsa_encode(msg_rsa, d, n)
print('Decrypted message')
print(msg_decode)
# convert_numbers_to_strings(msg_decode)

KeyboardInterrupt: 